In [1]:
from datetime import date,timedelta
import pandas as pd
import re
import numpy as np

In [2]:
def allSundays(year):
   d = date(year, 1, 1)
   d += timedelta(days = 6 - d.weekday())
   while d.year == year:
      yield d
      d += timedelta(days = 7)

In [3]:
allSundaysList = []
for d in allSundays(2017):
    allSundaysList.append(str(d).replace('-',''))

In [4]:
#constants or configuration settings
pd.set_option('display.float_format', lambda x: '%.8f' % x)
reCleanSplChars = re.compile(r'[a-zA-Z \t$%,?<>]*')

In [5]:
def GetDataAndProcess(weekendDate):
    #import pdb;pdb.set_trace()
    snapshotUrl = 'https://coinmarketcap.com/historical/{0}/'.format(weekendDate)
    snapshotCoinDf = pd.read_html(snapshotUrl)[0]
    snapshotCoinDf.rename(columns={'#':'RankNum'},inplace=True)
    
    snapshotCoinDf['PriceClean'] = snapshotCoinDf['Price'].str.replace(reCleanSplChars,'').str.replace(r'[0-9]*(-)[0-9]*','')
    snapshotCoinDf['Volume24hClean'] = snapshotCoinDf['Volume (24h)'].str.replace(reCleanSplChars,'')
    snapshotCoinDf['P7dClean'] = snapshotCoinDf['% 7d'].str.replace(reCleanSplChars,'')
    snapshotCoinDf['PriceClean'] = pd.to_numeric(snapshotCoinDf['PriceClean'])
    snapshotCoinDf['Volume24hClean'] = pd.to_numeric(snapshotCoinDf['Volume24hClean'])
    snapshotCoinDf['P7dClean'] = pd.to_numeric(snapshotCoinDf['P7dClean'])
    return(snapshotCoinDf)

In [6]:
def filterGroupNCalcData(cleanedDataFrame):
    tempDf = cleanedDataFrame[(cleanedDataFrame['PriceClean']>0.0001) & (cleanedDataFrame['PriceClean']<0.01)].copy()
    return(tempDf['P7dClean'].groupby(pd.cut(tempDf['Volume24hClean'],[100,1000,10000,tempDf.Volume24hClean.max()]
                                             ,labels=['100-1000','1000-10000','10000-MaxVol'])).mean())

In [7]:
masterListVar = []
#for iDay in allSundaysList:
for iDay in ['20170326']:
    t = 'tsDataFrame_'+iDay
    s = 'summaryDf_'+iDay
    dictDf = {}
    dictDf[t] = GetDataAndProcess(iDay).copy()
    dictDf[s] = filterGroupNCalcData(dictDf[t]).copy()
    ##tsDataFrame_+iDay = GetDataAndProcess(iDay).copy()
    ##summaryDf_+iDay = filterGroupNCalcData(tsDataFrame_+iDay).copy()
    masterListVar.append({'forDate':iDay,'dataSet':dictDf[t],'summaryDataSet':dictDf[s]})

In [45]:
excelWriterStream = pd.ExcelWriter('CoinsSummaryAnalysis.xlsx',engine='xlsxwriter')
finalSummaryDf = pd.DataFrame()
for mlv in masterListVar:
    #print(mlv['forDate']);
    #print(mlv['summaryDataSet'])
    mlv['dataSet'].to_excel(excelWriterStream,sheet_name=mlv['forDate'])
    mlv['summaryDataSet'].rename(mlv['forDate'],inplace=True)
    finalSummaryDf[mlv['forDate']] = mlv['summaryDataSet']

finalSummaryDf.to_excel(excelWriterStream,sheet_name='Summary')
excelWriterStream.save()

In [12]:
testMarchDf = pd.DataFrame(masterListVar[0]['dataSet']).copy()

In [14]:
masterListVar[0]['summaryDataSet']

Volume24hClean
100-1000        8.72200000
1000-10000     42.53666667
10000-MaxVol   42.06088235
Name: P7dClean, dtype: float64

In [16]:
testMarchDf[(testMarchDf['PriceClean']>0.0001) & (testMarchDf['PriceClean']<0.01)]

,RankNum,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d,PriceClean,Volume24hClean,P7dClean
3,4,XRP Ripple,XRP,"$353,371,097",$0.009451,"37,388,960,792 *","$9,880,955",0.39%,-7.69%,41.14%,0.00945100,9880955.00000000,41.14000000
17,18,DOGE Dogecoin,DOGE,"$32,195,923",$0.000296,108686658467,"$1,645,108",0.06%,14.05%,33.75%,0.00029600,1645108.00000000,33.75000000
28,29,XLM Stellar,XLM,"$14,721,350",$0.002072,"7,104,805,570 *","$641,797",0.25%,-1.20%,6.29%,0.00207200,641797.00000000,6.29000000
31,32,BTS BitShares,BTS,"$13,576,190",$0.005239,"2,591,243,958 *","$382,187",0.14%,-1.66%,15.61%,0.00523900,382187.00000000,15.61000000
35,36,SC Siacoin,SC,"$12,500,391",$0.000511,24448089084,"$246,787",0.85%,4.51%,36.98%,0.00051100,246787.00000000,36.98000000
54,55,ROUND Round,ROUND,"$5,648,484",$0.006645,"850,000,000 *","$54,621",0.19%,13.84%,11.48%,0.00664500,54621.00000000,11.48000000
65,66,BAY BitBay,BAY,"$3,486,959",$0.003461,"1,007,391,220 *","$49,595",-0.09%,22.15%,80.38%,0.00346100,49595.00000000,80.38000000
72,73,DGB DigiByte,DGB,"$2,486,168",$0.000307,8107031908,"$39,256",0.10%,-2.85%,23.51%,0.00030700,39256.00000000,23.51000000
75,76,BURST Burst,BURST,"$2,372,813",$0.001368,1734465449,"$50,046",0.27%,11.09%,34.30%,0.00136800,50046.00000000,34.30000000
77,78,VASH VPNCoin,VASH,"$2,260,103",$0.005645,"400,339,746 *","$136,906",1.24%,9.31%,12.74%,0.00564500,136906.00000000,12.74000000
